In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import tensorflow as tf
import time

NUM_AAS = 20
NUM_DIMENSIONS = 3

def masking_matrix(mask, name=None):
    """ Constructs a masking matrix to zero out pairwise distances due to missing residues or padding. 

    Args:
        mask: 0/1 vector indicating whether a position should be masked (0) or not (1)

    Returns:
        A square matrix with all 1s except for rows and cols whose corresponding indices in mask are set to 0.
        [MAX_SEQ_LENGTH, MAX_SEQ_LENGTH]
    """

    with tf.name_scope(name, 'masking_matrix', [mask]) as scope:
        mask = tf.convert_to_tensor(mask, name='mask')

        mask = tf.expand_dims(mask, 0)
        base = tf.ones([tf.size(mask), tf.size(mask)])
        matrix_mask = base * mask * tf.transpose(mask)

        return matrix_mask
        
def read_protein(filename_queue, max_length, num_evo_entries=21, name=None):
    """ Reads and parses a ProteinNet TF Record. 

        Primary sequences are mapped onto 20-dimensional one-hot vectors.
        Evolutionary sequences are mapped onto num_evo_entries-dimensional real-valued vectors.
        Secondary structures are mapped onto ints indicating one of 8 class labels.
        Tertiary coordinates are flattened so that there are 3 times as many coordinates as 
        residues.

        Evolutionary, secondary, and tertiary entries are optional.

    Args:
        filename_queue: TF queue for reading files
        max_length:     Maximum length of sequence (number of residues) [MAX_LENGTH]. Not a 
                        TF tensor and is thus a fixed value.

    Returns:
        id: string identifier of record
        one_hot_primary: AA sequence as one-hot vectors
        evolutionary: PSSM sequence as vectors
        secondary: DSSP sequence as int class labels
        tertiary: 3D coordinates of structure
        matrix_mask: Masking matrix to zero out pairwise distances in the masked regions
        pri_length: Length of amino acid sequence
        keep: True if primary length is less than or equal to max_length
    """

    with tf.name_scope(name, 'read_protein', []) as scope:
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(filename_queue)

        context, features = tf.parse_single_sequence_example(serialized_example,
                                context_features={'id': tf.FixedLenFeature((1,), tf.string)},
                                sequence_features={
                                    'primary':      tf.FixedLenSequenceFeature((1,),               tf.int64),
                                    'evolutionary': tf.FixedLenSequenceFeature((num_evo_entries,), tf.float32, allow_missing=True),
                                    'secondary':    tf.FixedLenSequenceFeature((1,),               tf.int64,   allow_missing=True),
                                    'tertiary':     tf.FixedLenSequenceFeature((NUM_DIMENSIONS,),  tf.float32, allow_missing=True),
                                    'mask':         tf.FixedLenSequenceFeature((1,),               tf.float32, allow_missing=True)})
        id_ = context['id'][0]
        primary =   tf.to_int32(features['primary'][:, 0])
        evolutionary =          features['evolutionary']
        secondary = tf.to_int32(features['secondary'][:, 0])
        tertiary =              features['tertiary']
        mask =                  features['mask'][:, 0]

        pri_length = tf.size(primary)
        keep = pri_length <= max_length

        one_hot_primary = tf.one_hot(primary, NUM_AAS)

        # Generate tertiary masking matrix--if mask is missing then assume all residues are present
        mask = tf.cond(tf.not_equal(tf.size(mask), 0), lambda: mask, lambda: tf.ones([pri_length]))
        ter_mask = masking_matrix(mask, name='ter_mask')        

        return id_, one_hot_primary, evolutionary, secondary, tertiary, ter_mask, pri_length, keep

C:\Users\Michal\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
def dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

def tf_rad2deg(rad):
    pi_on_180 = 0.017453292519943295
    return rad / pi_on_180

# takes 1 dimensional tensor and outputs an angle
def dihedral_tf1(p):
    p0 = tf.gather(p, 0)
    p1 = tf.gather(p, 1)
    p2 = tf.gather(p, 2)
    p3 = tf.gather(p, 3)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1))
    
    v = tf.subtract(b0, tf.multiply(tf.tensordot(b0, b1, 1), b1))
    w = tf.subtract(b2, tf.multiply(tf.tensordot(b2, b1, 1), b1))
    
    x = tf.tensordot(v, w, 1)
    y = tf.tensordot(tf.cross(b1, v), w, 1)
    
    return tf_rad2deg(tf.atan2(y,x))

# takes 2 dimensional tensor (K, 4) and outputs K angles
def dihedral_tf2(p):
    p0 = tf.gather(p, 0, axis=1)
    p1 = tf.gather(p, 1, axis=1)
    p2 = tf.gather(p, 2, axis=1)
    p3 = tf.gather(p, 3, axis=1)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.norm(b1, axis=0))
    
    v = tf.subtract(b0, tf.multiply(tf.tensordot(b0, b1, 2), b1))
    w = tf.subtract(b2, tf.multiply(tf.tensordot(b2, b1, 2), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 1, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 1, keepdims=True )

    return tf_rad2deg(tf.atan2(y,x))

# takes a 3 dimensional tensor (N, K, 4) and outputs (N,K) angles
def dihedral_tf3(p):
    p0 = tf.gather(p, 0, axis=2)
    p1 = tf.gather(p, 1, axis=2)
    p2 = tf.gather(p, 2, axis=2)
    p3 = tf.gather(p, 3, axis=2)
    
    b0 = -1.0 * (tf.subtract(p1, p0))
    b1 = tf.subtract(p2, p1)
    b2 = tf.subtract(p3, p2)
    
    b1 = tf.divide(b1, tf.expand_dims(tf.norm(b1, axis=1), axis=1))
    
    v = tf.subtract(b0, tf.einsum('b,bij->bij', tf.einsum('bij,bij->b', b0, b1), b1))
    w = tf.subtract(b2, tf.einsum('b,bij->bij', tf.einsum('bij,bij->b', b2, b1), b1))
    
    x = tf.reduce_sum( tf.multiply( v, w ), 2, keepdims=True )
    y = tf.reduce_sum( tf.multiply( tf.cross(b1, v), w ), 2, keepdims=True )

    return tf_rad2deg(tf.atan2(y,x))

def slice_tf(tensor):
    return tf.slice(tensor, (0,0), (4,3))

In [416]:
with tf.Session() as sess:
    out = sess.run([slice_tf(test1)])
    
np.array(out).shape

(1, 4, 3)

We need 4 coordinates to calculate an angle but the proteins are organized in 3x3 matricies. Thus how do we calculate an angle. Take 3 angles from 1 aminoacid and 1 angle from the next?

In [445]:

p1 = np.array([[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ],[
                [ 1,           0,         0     ],
                [ 0,           0,         0     ],
                [ 0,           0,         1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ],
                [ 0.999999,    0.000001,  1     ]
            ]])
print("p1shape", p1[0].shape)
p1_tf = tf.convert_to_tensor(p1[0])
test1 = test[0]
test_full

r = test1.shape[0]
n = 4
a_list = list(range(r))
the_list1 = np.array([a_list[slice(i, i+n)] for i in range(r - n+1)])
print(the_list1.shape, test1.shape)

r = test_full.shape[1]
n = 4
a_list = list(range(r))
the_list2 = [a_list[slice(i, i+n)] for i in range(r - n+1)]
the_list2 = np.array([the_list2 for _ in range(test_full.shape[0])])
the_list2 = the_list2
# print(the_list2.shape, test_full.shape)
# for i in range(len(r) - n + 1):
#     r[i: i + n]

p1_tf_stacked_full = tf.stack(tf.gather(test_full, the_list1, axis=1))

angle1 = dihedral_tf1(p1_tf)
angle2 = dihedral_tf2(p1_tf_stacked)
angle3 = dihedral_tf3(p1_tf_stacked_full)

with tf.Session() as sess:
    p1_tf_stacked_, p1_tf_stacked_full_, angle1_, angle2_, angle3_ = sess.run([p1_tf_stacked, p1_tf_stacked_full, angle1, angle2, angle3])

dihedral(p1[0]), np.array(p1_tf_stacked_).shape, np.array(p1_tf_stacked_full_).shape, angle1_, angle2_.shape, angle3_.shape

p1shape (7, 3)
(2445, 4) (2448, 3)


ValueError: Fetch argument <tf.Tensor 'Gather:0' shape=(2445, 4, 3) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("Gather:0", shape=(2445, 4, 3), dtype=float32) is not an element of this graph.)

In [424]:
array = np.array([[0, 1],[1, 2],[2, 3],[3, 4],[4, 5],[5, 6]])

r = array.shape[0]
n = 4
a_list = list(range(r))
the_list = np.array([a_list[slice(i, i+n)] for i in range(r - n+1)])

array[the_list].shape

print(array[the_list])

[[[0 1]
  [1 2]
  [2 3]
  [3 4]]

 [[1 2]
  [2 3]
  [3 4]
  [4 5]]

 [[2 3]
  [3 4]
  [4 5]
  [5 6]]]


In [378]:
angle2_[36], angle3_[0][36]

(array([141.95627], dtype=float32), array([141.95627], dtype=float32))

In [299]:
angle2_[0].shape, angle2_[1].shape, angle3_[0].shape, angle3_[1].shape

((2445, 3), (2445, 3), (2, 2445, 3), (2, 2445, 3))

In [301]:
np.einsum('bij,bij->b', angle3_[0], angle3_[1])

array([-756.6741, -756.6741], dtype=float32)

In [31]:
tf.reset_default_graph()

num_epochs = 1
a_path = r'C:\Users\Michal\Desktop\ITU NLP\casp7\training\30\*'
# init_op = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())
base_names = glob.glob(a_path)
base_tensor = tf.convert_to_tensor(base_names[:1])
file_queue = tf.train.string_input_producer(
    base_tensor,
    num_epochs=num_epochs,
    shuffle=False # Note: must set shuffle to False
)

res = read_protein(file_queue, max_length=1000)
id_, one_hot_primary, evolutionary, secondary, tertiary, ter_mask, pri_length, keep = res
# one_hot_primary = tf.slice(one_hot_primary, 0, 10)

lstm_units = 5
batch_size=8
capacity=1000
min_after_dequeue=100

## I couldn't make shuffle batch work
## because it doesn't have the dynamic padding included
# ids, data, length = tf.train.shuffle_batch(
#       [id_, one_hot_primary, pri_length], 
#       batch_size=batch_size, 
#       capacity=capacity,
#       min_after_dequeue=min_after_dequeue)

# dynamic pad makes sure that the length of the proteins
# is padded to the longest protein in the batch
ids, data, labels, length = tf.train.batch(
      [id_, one_hot_primary, tertiary, pri_length], 
      batch_size=batch_size, 
      capacity=capacity, 
      dynamic_pad=True
    )

protein_length = tf.gather(tf.shape(data), 1)
protein_euc_length = tf.gather(tf.shape(labels), 1)

# r = test1.shape[1]
# n = 4
# a_list = list(range(r))
# the_list1 = np.array([a_list[slice(i, i+n)] for i in range(r - n+1)])
the_list = tf.placeholder(tf.int32, shape=(None, 4))

# tile_test = tf.tile([tf.reduce_max(length)], (batch_size,))
p1_tf_stacked_full = tf.stack(tf.gather(labels, the_list, axis=1))
angle3 = dihedral_tf3(p1_tf_stacked_full)

padding = tf.constant([[0, 0], [0,3], [0,0]])
angle3 = tf.pad(angle3, padding)

angle3_shape = tf.gather(tf.shape(angle3), [0,1])
angle3 = tf.reshape(angle3, shape=angle3_shape)
angle3 = tf.reshape(angle3, shape=(tf.gather(angle3_shape, 0), protein_length, 3))


# seq_length = tf.reduce_max(length)
cell = tf.nn.rnn_cell.LSTMCell(num_units=lstm_units, state_is_tuple=True)
outputs, states = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=cell,
    cell_bw=cell,
    dtype=tf.float32,
    inputs=data)
outputs_conc = tf.concat(outputs, 2)
pred = tf.layers.dense(outputs_conc, 3, activation=tf.nn.tanh, use_bias=False)
# loss = tf.losses.mean_squared_error(labels=labels, predictions=pred)

In [33]:
num_examples = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()  
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
#     batch = tf.train.shuffle_batch([id_], 10, 200, 100)
    try:
        step = 0
        while not coord.should_stop():
            start_time = time.time()
            
            h = sess.partial_run_setup([protein_euc_length, protein_length, angle3, p1_tf_stacked_full, labels], [the_list])
        
            protein_euc_length_, protein_length_ = sess.partial_run(h, [protein_euc_length, protein_length])
#             protein_euc_length_ = protein_euc_length_
#             protein_length_ = protein_length_[0]
            print('protein_euc_length', protein_euc_length_)
            print('protein_length', protein_length_)
        
#             r = array.shape[0]
            n = 4
            a_list = list(range(protein_euc_length_))
            the_list_ = np.array([a_list[slice(i, i+n)] for i in range(protein_euc_length_ - n+1)])
            print(the_list_.shape)
            
            angle3_, p1_tf_stacked_full_, labels_ = sess.partial_run(h, [angle3, 
                                                                         p1_tf_stacked_full, labels], 
                                                                         feed_dict={the_list: the_list_})
            
            print(np.array(angle3_).shape)

            duration = time.time() - start_time

    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (num_epochs, step))
#         print(num_examples)
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

protein_euc_length 1143
protein_length 381
(1140, 4)
(8, 381, 3)
protein_euc_length 1167
protein_length 389
(1164, 4)
(8, 389, 3)
protein_euc_length 1095
protein_length 365
(1092, 4)
(8, 365, 3)
protein_euc_length 831
protein_length 277
(828, 4)
(8, 277, 3)
protein_euc_length 921
protein_length 307
(918, 4)
(8, 307, 3)
protein_euc_length 2574
protein_length 858
(2571, 4)
(8, 858, 3)
protein_euc_length 1785
protein_length 595
(1782, 4)
(8, 595, 3)
protein_euc_length 1497
protein_length 499
(1494, 4)
(8, 499, 3)
protein_euc_length 825
protein_length 275
(822, 4)
(8, 275, 3)
protein_euc_length 1623
protein_length 541
(1620, 4)
(8, 541, 3)
protein_euc_length 969
protein_length 323
(966, 4)
(8, 323, 3)
protein_euc_length 1302
protein_length 434
(1299, 4)
(8, 434, 3)
protein_euc_length 1824
protein_length 608
(1821, 4)
(8, 608, 3)
protein_euc_length 804
protein_length 268
(801, 4)
(8, 268, 3)
protein_euc_length 1512
protein_length 504
(1509, 4)
(8, 504, 3)
protein_euc_length 930
protein_leng

In [ ]:
# num_examples = 0
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.local_variables_initializer())
#     coord = tf.train.Coordinator()  
#     threads = tf.train.start_queue_runners(coord=coord, sess=sess)
# #     batch = tf.train.shuffle_batch([id_], 10, 200, 100)
#     try:
#         step = 0
#         while not coord.should_stop():
#             start_time = time.time()
# #             ids_, data_, labels_, length_ = sess.run([ids, data, labels, length])
            
# #             out = sess.run(outputs)
#             test, length_, outputs_, outputs_conc_, pred_ = sess.run([tile_test, length, outputs, outputs_conc, pred])
# #             print(outputs_[0].shape, outputs_[1].shape, outputs_conc_.shape, pred_.shape, loss)
            
#             labels_ = sess.run([labels])
            
#             print(np.array(labels_).shape, np.array(labels_)[0][:2][:9])
#             test = np.array(labels_)[0][:2]
# #             print(ids_.shape, data_.shape, length_)
# #             print "grabbing"
# #             e, l = sess.run([example_batch, label_batch])
# #             num_examples = num_examples + ids_.shape[0]
# #             print "num_examples = " + str(num_examples)
#             duration = time.time() - start_time

#     except tf.errors.OutOfRangeError:
#         print('Done training for %d epochs, %d steps.' % (num_epochs, step))
# #         print(num_examples)
#     finally:
#         # When done, ask the threads to stop.
#         coord.request_stop()

#         # Wait for threads to finish.
#         coord.join(threads)
#         sess.close()